In [1]:
import pandas as pd
from io import BytesIO
from bs4 import BeautifulSoup
import requests

In [49]:
url = 'https://www.billboard.com/charts/hot-100'
response = requests.get(url)

In [122]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
import time
import os
import xlsxwriter

In [63]:
options = Options()
# options.headless = True
browser = webdriver.Chrome(options=options)
browser.implicitly_wait(3)
browser.get(url)
time.sleep(40) 
# BeautifulSoup을 활용하여 데이터 파싱
html = browser.page_source
soup = BeautifulSoup(html, 'html.parser')

In [64]:
#<span class="chart-element__information__song text--truncate color--primary">For The Night</span>
songs = soup.findAll('span', {'class' : 'chart-element__information__song'})

In [220]:
songlist = []
for song in songs:
    print(song.text)
    songlist.append(song.text)

Rockstar
Blinding Lights
Whats Poppin
Savage
Roses
For The Night
Watermelon Sugar
Blueberry Faygo
Say So
Intentions
The Woo
Adore You
Circles
We Paid
Don't Start Now
Rain On Me
Break My Heart
The Bones
One Margarita
The Bigger Picture
The Box
Dior
I Hope
Life Is Good
Falling
Before You Go
Hard To Forget
Go Crazy
Savage Love (Laxed - Siren Beat)
Sunday Best
Got It On Me
Death Bed
Bluebird
Aim For The Moon
Chasin' You
Party Girl
Gangstas
Toosie Slide
44 Bulldog
High Fashion
Something Special
Die From A Broken Heart
Yea Yea
Mood Swings
Stuck With U
Emotionally Scarred
Supalonely
Walk Em Down
Make It Rain
Be Kind
Girls In The Hood
What You Know Bout Love
The Scotts
Snitchin
Bad Bitch From Tokyo (Intro)
Enjoy Yourself
Creature
I Love My Country
If The World Was Ending
Got What I Got
Like That
Here And Now
Said Sum
Rags2Riches
West Coast Shit
Be A Light
Mamacita
Don't Rush
Done
P*$$y Fairy (OTW)
Does To Me
God Whispered Your Name
One Big Country Song
Life's A Mess
Wash Us In The Blood
Diana


In [138]:
# 
songlist[69]='Pretty Fairy'

In [67]:
#<span class="chart-element__image flex--no-shrink" style="background-image: url(&quot;https://charts-static.billboard.com/img/2020/03/megan-thee-stallion-z0z-savage-7q2-155x155.jpg&quot;);"></span>
images = soup.findAll('span', {'class' : 'chart-element__image'})

In [142]:
img_dir = f"{os.getcwd()}/billboard"
if not os.path.isdir(img_dir):
   os.mkdir(img_dir)

i=0
for image in images :
    url = image.get('style').split('"')[-2]   
    img_file_name = songlist[i]
    i+=1
    if i ==100:
        break
    img_data = requests.get(url).content
    with open(f'billboard/{i}_{img_file_name}.jpg', 'wb') as handler:
        handler.write(img_data)

In [221]:
# 날짜 받아오기
from datetime import datetime

now = datetime.now().strftime('%Y-%m-%d')
print(now)

2020-07-21


In [216]:
# 19번 곡이 사이즈가 너무 큼...
from PIL import Image

i = 18
image = 'billboard/'+str(i+1)+'_'+songlist[i]+'.jpg'

im = Image.open(image)
print(im.size)

resize_image = im.resize((155, 155))
resize_image.save(image, "JPEG", quality=95 )

(155, 155)


In [217]:
# 엑셀에 저장
name = 'Billboard_'+now+'.xlsx'
workbook  = xlsxwriter.Workbook(name)
worksheet = workbook.add_worksheet()

for i in range(len(songlist)):    
    worksheet.set_column('A:A', 35)
    worksheet.set_column('B:B',20)
    worksheet.set_default_row(110)
    
    worksheet.write('A'+str(i+1), songlist[i])
    
    image = 'billboard/'+str(i+1)+'_'+songlist[i]+'.jpg'
    worksheet.insert_image('B'+ str(i+1),image)

workbook.close()